In [2]:
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.oauth2.credentials import Credentials
from google.oauth2 import service_account
from google.auth.transport.requests import Request
import os.path
import json
import pickle

def oauth():
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets', 
              'https://www.googleapis.com/auth/script.projects', 'https://www.googleapis.com/auth/drive', 'https://www.googleapis.com/auth/drive.file'
              ]
    creds_filename = 'test-daebong-service-account.json'  # 서비스 계정 파일 경로를 지정합니다.

    # 서비스 계정 파일을 사용하여 인증 정보를 로드합니다.
    creds = service_account.Credentials.from_service_account_file(creds_filename, scopes=SCOPES)
    return creds

def oauthByWeb():
    # 필요한 스코프를 지정합니다.
    SCOPES = [
        'https://www.googleapis.com/auth/spreadsheets',
        'https://www.googleapis.com/auth/script.projects',
        'https://www.googleapis.com/auth/drive',
        'https://www.googleapis.com/auth/drive.file'
    ]
    creds = None
    # 'token.json' 파일이 존재하면, 저장된 인증 정보를 불러옵니다.
    if os.path.exists('token.json'):
        with open('token.json', 'rb') as token:
            creds = pickle.load(token)

    # 저장된 인증 정보가 없거나, 유효하지 않은 경우 새로운 인증을 진행합니다.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            creds_filename = 'oauth-new-daebong.json'
            flow = InstalledAppFlow.from_client_secrets_file(creds_filename, SCOPES)
            creds = flow.run_local_server(port=8080)
        # 새로운 인증 정보를 'token.json'에 저장합니다.
        with open('token.json', 'wb') as token:
            pickle.dump(creds, token)

    return creds

In [3]:
def get_spreadsheet_data_from_drive(service, folder_id):
    # 폴더 내 스프레드 시트 목록 조회
    query = f"'{folder_id}' in parents and mimeType = 'application/vnd.google-apps.spreadsheet' and trashed = false"
    spreadsheets = []
    page_token = None

    while True:
        results = service.files().list(q=query,
                                       spaces='drive',
                                       fields='nextPageToken, files(id, name)',
                                       orderBy='name',
                                       pageToken=page_token).execute()
        items = results.get('files', [])
        page_token = results.get('nextPageToken', None)

        if not items:
            print('No files found.')
            break
        else:
            for item in items:
                # 각 스프레드 시트의 URL을 생성합니다.
                spreadsheet_url = f"https://docs.google.com/spreadsheets/d/{item['id']}"
                print(f"Name: {item['name']}, ID: {item['id']}, URL: {spreadsheet_url}")
                spreadsheets.append({
                    'name': item['name'],
                    'id': item['id'],
                    'url': spreadsheet_url
                })

        if page_token is None:
            break

    return spreadsheets

In [36]:
def get_filtered_spreadsheet_data(service, folder_id):
    # 폴더 내 스프레드 시트 목록 조회
    query = f"'{folder_id}' in parents and mimeType = 'application/vnd.google-apps.spreadsheet' and trashed = false"
    filtered_data = []
    page_token = None

    while True:
        results = service.files().list(q=query,
                                       spaces='drive',
                                       fields='nextPageToken, files(id, name)',
                                       orderBy='name',
                                       pageToken=page_token).execute()
        items = results.get('files', [])
        page_token = results.get('nextPageToken', None)

        for item in items:
            name = item['name']
            # "_" 포함 여부 확인
            user_id = name
            user_name = ""

            if "_" in name:
                user_id = name.split("_")[0]  # "_" 기준으로 앞의 값을 가져옴
                user_name = name.split("_")[1]
            try:
                # 숫자로 변환 가능한지 확인
                num = int(user_id)
                id_data = item['id']
                # 조건을 만족하면 데이터 추가
                spreadsheet_url = f"https://docs.google.com/spreadsheets/d/{item['id']}"
                spreadsheet_url_data = spreadsheet_url
                filtered_data.append([num, id_data, spreadsheet_url_data, user_name])

            except ValueError:
                # 숫자로 변환 불가능한 경우 무시
                continue

        if page_token is None:
            break
    # 104 예외처리    
    filtered_data.append([104, "", "", ""])
    # 숫자 기준으로 정렬하여 반환
    filtered_data = [data for data in filtered_data if 76 <= data[0] <= 130] 
    filtered_data.sort(key=lambda x: x[0])
    return filtered_data

In [40]:
def update_sheet_byID_by_url(service, spreadsheet_id, sheet_name, data, E_start_range):
    # 데이터 리스트
    # E열과 F열에 삽입할 값 준비
    values = [[item[1], item[2]] for item in data]  # 각 요소의 인덱스 1번과 2번 값을 사용
    body = {
        'values': values
    }

    # Google Sheets API를 사용하여 데이터 쓰기
    range_name = f"{sheet_name}!E{E_start_range}:F{E_start_range + len(values) + 1}"
    request = service.spreadsheets().values().update(spreadsheetId=spreadsheet_id, range=range_name,
                                                    valueInputOption="RAW", body=body)
    response = request.execute()

    print("Data has been written successfully.")

In [41]:
#전체 발주서 폴더
folder_id = "1P3fb9h8Owhm2AzFdJ7bDH3ZN4psbBh2c"
#민기 자동화 발주서 폴더
# folder_id = "16xozEJ0fwXmbnJORnRXfi5y_Gq3PwScH"
spreadsheet_id = "1Ajz2g-pChLSoies5umhPMcgTp5m2MIjhljJOviPlWCU"
sheet_name = "셀러_시트ID"


if __name__ == "__main__":
    creds = oauthByWeb()
    service_sheets = build('sheets', 'v4', credentials=creds)
    service_drive = build("drive", "v3", credentials=creds)
    
    # items = get_spreadsheet_data_from_drive(service_drive, folder_id)
    data = get_filtered_spreadsheet_data(service_drive, folder_id)
    print(data)
    print(len(data))
    update_sheet_byID_by_url(service_sheets, spreadsheet_id, sheet_name, data, 56)
    


[[76, '1RTS4gFbGpELMAoXn6gEq30U8Gxsw9sc759XAeLIalIo', 'https://docs.google.com/spreadsheets/d/1RTS4gFbGpELMAoXn6gEq30U8Gxsw9sc759XAeLIalIo', '안서현'], [77, '1UfwIKFVNNRT8VjN-hP_YWirS04TguC6HQqHTAYx770k', 'https://docs.google.com/spreadsheets/d/1UfwIKFVNNRT8VjN-hP_YWirS04TguC6HQqHTAYx770k', '계절의과일'], [78, '1gOsaEoEtS8z7HTiyi4GHWiPyt1SwxNrd55AzzcD02x4', 'https://docs.google.com/spreadsheets/d/1gOsaEoEtS8z7HTiyi4GHWiPyt1SwxNrd55AzzcD02x4', '살가운청과'], [79, '1-BzndUe5rc7DIU74Q6IsVqQ4yGMnxnF5ge0Pz1JsjKI', 'https://docs.google.com/spreadsheets/d/1-BzndUe5rc7DIU74Q6IsVqQ4yGMnxnF5ge0Pz1JsjKI', '달곰한'], [80, '17aRoa0ZjpEMeJ9msNvlutnonCQlOAZgWiul1K24JAk4', 'https://docs.google.com/spreadsheets/d/17aRoa0ZjpEMeJ9msNvlutnonCQlOAZgWiul1K24JAk4', '선희네농수산'], [81, '1wYgtTYSHcq7YAm8XMgR-RmYDFtgAaKp684lK1eKfKB8', 'https://docs.google.com/spreadsheets/d/1wYgtTYSHcq7YAm8XMgR-RmYDFtgAaKp684lK1eKfKB8', '꼬기스토어'], [82, '1S3HcXMgbfe3vADUn7Tay55RCocrSfAwkMQKHhroei90', 'https://docs.google.com/spreadsheets/d/1S3HcXMgb

In [21]:
count = 0
array_num = [element[0] for element in data]
array_none = []

for i in range(22, 226):
    if i in array_num:
        print(i)
        count += 1

    else:
        array_none.append(i)

print(count, array_none, len(array_none))

22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
204 [] 0
